In [1]:
import cv2
import numpy as np
np.set_printoptions(threshold=np.inf)
from matplotlib import pyplot as plt
from os.path import isfile, join
from os import listdir
from sklearn import preprocessing

# TASK 1 - Compute image descriptors (QS1)

### Calcular els histogrames de totes les imatges de la carpeta especificada a "imagesFolder"
Aquí la idea es fer el bucle que recorre totes les imatges i calcula el histograma de cadascuna d'elles, afegint el resultat a una matriu d'histogrames que després ens servirán per calcular les "Similarity measures".
Cada fila correspondrà a una imatge. 

In [2]:
imagesFolder = "./bbdd/"
histogram_bbdd_matrix = np.empty([0, 256*3]) #Creem una matriu buida
print(histogram_bbdd_matrix.shape)
for filename in sorted(listdir(imagesFolder)):
    if(filename != '.DS_Store' ):
        print("******************************************************************************")
        print("IMAGE: " + filename)
        print("******************************************************************************")

        img = cv2.imread(imagesFolder + filename)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2Lab)
        color = ('x','y','z')
        hist_img = np.empty([0,0])
        for i,col in enumerate(color):
            hist = cv2.calcHist([img],[i],None,[256],[0,256]) #Calculem histogrames
            hist = preprocessing.normalize(hist, norm='l2') #Normalitzem histogrames
            hist_t = hist.transpose()
            print(hist_t.shape)
            if i == 0:
                hist_img = hist_t
            else:
                hist_img = np.concatenate((hist_img, hist_t), axis = 1)
            
        histogram_bbdd_matrix = np.vstack((histogram_bbdd_matrix, hist_img))   

(0, 768)
******************************************************************************
IMAGE: bbdd_00000.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00001.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00002.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00003.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00004.jpg
**************************************************************

(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00041.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00042.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00043.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00044.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00045.jpg
********************************************

(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00091.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00092.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00093.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00094.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00095.jpg
********************************************

(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00133.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00134.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00135.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00136.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00137.jpg
********************************************

(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00179.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00180.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00181.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00182.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00183.jpg
********************************************

(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00222.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00223.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00224.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00225.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00226.jpg
********************************************

(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00267.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00268.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00269.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00270.jpg
******************************************************************************
(1, 256)
(1, 256)
(1, 256)
******************************************************************************
IMAGE: bbdd_00271.jpg
********************************************

In [3]:
print(histogram_bbdd_matrix.shape)

(279, 768)


### Calcular els histogrames de totes les imatges de la carpeta de query (qsd1_w1)

In [4]:
queryFolder = "./qsd1_w1/"
histogram_query_matrix = np.empty([0, 256*3])
for filename in sorted(listdir(queryFolder)):
    if(filename != '.DS_Store' and (filename.split('.')[1] == 'jpg' or filename.split('.')[1] == 'png')):
        print("******************************************************************************")
        print("./qsd1_w1/" + filename)
        print("******************************************************************************")

        query_img = cv2.imread(queryFolder + filename)
        query_img = cv2.cvtColor(query_img,cv2.COLOR_BGR2Lab)
        color = ('b','g','r')
        hist_query = np.empty([0, 256*3])
        
        for i,col in enumerate(color):
            histr = cv2.calcHist([query_img],[i],None,[256],[0,256])
            histr = preprocessing.normalize(histr, norm='l2')
            histr = histr.transpose()
            if i == 0:
                hist_query = histr
            else:
                hist_query = np.concatenate((hist_query,histr), axis = 1)
        
        histogram_query_matrix = np.vstack((histogram_query_matrix, hist_query))
    


******************************************************************************
./qsd1_w1/00000.jpg
******************************************************************************
******************************************************************************
./qsd1_w1/00001.jpg
******************************************************************************
******************************************************************************
./qsd1_w1/00002.jpg
******************************************************************************
******************************************************************************
./qsd1_w1/00003.jpg
******************************************************************************
******************************************************************************
./qsd1_w1/00004.jpg
******************************************************************************
******************************************************************************
./qsd1_w1/00005.jpg
***********

In [5]:
print(histogram_query_matrix.shape)

(30, 768)


# TASK 2 - Similarity measures

### Euclidean distance

In [6]:
from scipy.spatial import distance

dst = np.zeros((30, 279))
matrix_retrieval = np.zeros((30,10))

for query_image_index in range(0,len(histogram_query_matrix)): 
    for bbdd_image_index in range(0, len(histogram_bbdd_matrix)):
        
        #Euclidian distance 1
        #dst[query_image_index,bbdd_image_index] = distance.euclidean(histogram_query_matrix[query_image_index,:], histogram_bbdd_matrix[bbdd_image_index,:])
        
        #Euclidian distance 2
        dst[query_image_index,bbdd_image_index] = np.linalg.norm(histogram_query_matrix[query_image_index,:] - histogram_bbdd_matrix[bbdd_image_index,:], ord=1)
        
    
    matrix_retrieval[query_image_index,:] = np.argsort(dst[query_image_index,:])[:10]

print(matrix_retrieval)

[[131. 173.   7. 154. 140. 162. 167.  81.  91. 132.]
 [125. 116. 133. 140. 157. 207. 236. 154. 261. 132.]
 [165. 207. 172. 157.  89. 187. 237.  81.  15. 149.]
 [157.  89.  31. 128. 180. 172.   4. 207.  70. 187.]
 [147. 184. 161.  56. 265.  87. 171.  46. 227. 136.]
 [ 83. 271. 246. 136. 158. 142.  90. 127. 180.  84.]
 [180.  60.  84. 198. 149. 245. 172. 240. 204. 255.]
 [181. 238.  95. 252. 204. 198. 253.  18.  33. 266.]
 [ 73. 144. 123. 148. 268. 215. 138.  20. 191. 109.]
 [ 82.  59.  32.  12. 137. 228.  58.  44. 191.   3.]
 [244. 205. 264.  80. 232.  70.  30. 177.  83.  63.]
 [205.  70. 177. 232. 157. 176. 207.  89.  30. 179.]
 [125. 133. 179.  19. 232.  81. 261. 205. 173. 116.]
 [ 73. 144.  46. 138. 123. 268. 148. 215. 184. 147.]
 [ 13. 223. 186. 114. 213. 267. 242.  68. 152. 196.]
 [189.  98.  48.  47.  37. 212. 247. 103. 269. 119.]
 [150.  49. 187. 161. 208.  66.  75. 171. 188. 237.]
 [194.  18.  30.  31. 128.  70.  60. 117.   4. 216.]
 [ 70.   4. 158. 187. 232. 180. 176. 128.  60.

# Task 4

In [7]:
import ml_metrics as metrics
import pickle

with open('./qsd1_w1/gt_corresps.pkl', 'rb') as fd:
        ll = pickle.load(fd)
        print(ll)

gt = np.empty((0,0))

#Convertir idx a list of lists
matrix_retrieval_lst = matrix_retrieval.tolist()

ll_prp = np.zeros((len(ll),1))

for i in range(0,len(ll)):
    ll_prp[i] = ll[i][0][1]

ll_prp_lst = ll_prp.tolist()

mapak = metrics.average_precision.mapk(ll_prp_lst, matrix_retrieval_lst, k=10)        

print(mapak)

[[[0, 94]], [[1, 116]], [[1, 122]], [[3, 180]], [[4, 147]], [[5, 271]], [[6, 106]], [[7, 193]], [[8, 73]], [[9, 59]], [[10, 5]], [[11, 205]], [[12, 14]], [[13, 73]], [[14, 178]], [[15, 189]], [[16, 150]], [[17, 160]], [[18, 176]], [[19, 215]], [[20, 228]], [[21, 62]], [[22, 266]], [[23, 33]], [[24, 272]], [[25, 184]], [[26, 92]], [[27, 218]], [[28, 131]], [[29, 279]]]
0.43642857142857144
